#Read dataset from bronze

In [0]:
df = spark. table("workspace.bronze.crm_prd")
df.display()

#Transformation

##remove unwanted spaces

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.functions import col, trim, StringType,regexp_replace



In [0]:
for field in df.schema.fields:
  if isinstance(field.dataType, StringType):
    df.withColumn(field.name, trim(col(field.name))).display()
 
    

##product key parsing

In [0]:
from pyspark.sql.functions import  length,regexp_replace

In [0]:
df=df.withColumn("cat_id",F.regexp_replace(F.substring(col("prd_key"),1,5),"-","_"))\
     .withColumn("prd_key",F.substring(col("prd_key"),7, F.length(col("prd_key"))))

df.display()


## cost clean

In [0]:
from pyspark.sql.functions import lit

In [0]:
df = df.withColumn("prd_cost",F.coalesce(col("prd_cost"),lit(0)))
df.display()


Date cast

In [0]:
df = df.withColumn("prd_start_dt",col("prd_start_dt").cast("date"))\
.withColumn("prd_end_dt",col("prd_end_dt").cast("date"))
df.display()



##rename column

In [0]:
colm = {
  "prd_id": "product_id",
  "prd_key": "product_key",
  "prd_nm": "product_name",
  "prd_cost": "product_cost",
  "prd_line": "product_line",
  "prd_start_dt": "start_date",
  "prd_end_dt": "end_date",
  "cat_id": "category_id",
 
}
for old, new in colm.items():
  df=df.withColumnRenamed(old, new)
  

In [0]:
df.display()

#Writing dataset in silver table

In [0]:
df.write.mode("overwrite").saveAsTable("workspace.silver.crm_prd")

##checks dataset

In [0]:
%sql
select*from workspace.silver.crm_prd